In [ ]:
#JSON Eater

In [1]:
########################################################################################################################
import pandas as pd
import json
from typing import List

def jsonToData(fileName:str) -> dict:
    ''' returns data for json as dict. '''
    with open(fileName, encoding='utf-8') as f:
        data = json.load(f)
    return data

def j2pd_playlists(fileName:str) -> dict:
    ''' returns a dict of DF's for each playlist. '''
    outp = {}
    pl_data = jsonToData(fileName)
    for i in pl_data['playlists']:
        df_pl = pd.DataFrame(columns = ['trackName','artistName','albumName'])
        pl_name = i['name']
        for j in i['items']:
            track = j['track']
            df_pl = pd.concat([df_pl, pd.DataFrame.from_dict([track])], ignore_index=True)
        outp[pl_name] = df_pl
    return outp

def j2pd_saved_tracks(sp, limit:int, offset:int):
    ''' max limit = 50 '''
    data = sp.current_user_saved_tracks(limit = limit, offset = offset)
    outp = pd.DataFrame(columns=['added_at','album_name','album_id','artists','duration_ms','id','name','popularity'])
    inpt = {}
    for i in data['items']:
        inpt['added_at'] = i['added_at']
        trk = i['track']
        inpt['album_name'] = trk['album']['name']
        inpt['album_id'] = trk['album']['id']
        arts = {}
        for j in trk['artists']:
            arts[j['id']]=j['name']
        inpt['artists'] = arts
        inpt['duration_ms'] = trk['duration_ms']
        inpt['id'] = trk['id']
        inpt['name'] = trk['name']
        inpt['popularity'] = trk['popularity']
        outp = pd.concat([outp, pd.DataFrame.from_dict([inpt])], ignore_index=True)
    return outp

In [ ]:
lib_file = 'SpotifyData/YourLibrary.json'
pl_file = 'SpotifyData/Playlist1.json'

In [2]:
#client
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os

def init_cred(cid):
    '''client credentials flow. promtps for secret. returns sp object'''
    secret = input("Give secret ID: ")
    if secret:
        os.environ['SPOTIPY_CLIENT_ID']= cid
        os.environ['SPOTIPY_CLIENT_SECRET']= secret
        client_credentials_manager = SpotifyClientCredentials()
        return spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    else:
        raise ValueError('Need a secret ID.')

In [ ]:
#join history files
#target: df_hist.columns = [id, msplayed, endtime]
hist0_file = 'SpotifyData/StreamingHistory0.json'
hist1_file = 'SpotifyData/StreamingHistory1.json'
hist2_file = 'SpotifyData/StreamingHistory2.json'
df_hist = pd.DataFrame(columns = ['endTime', 'artistName', 'trackName', 'msPlayed'])
for i in range(0,3):
    fileName = ''.join(['SpotifyData/StreamingHistory', str(i), '.json'])
    data = jsonToData(fileName)
    df_hist = pd.concat([df_hist, pd.DataFrame.from_dict(data)], ignore_index=True)
df_hist.columns = ['endTime', 'artist_name', 'name', 'msPlayed']

In [3]:
sp = init_cred('73dfaea6d85240e8bf6e69b48e12339e')

In [ ]:
q = ''.join(['artist:', 'Pegboard Nerds', ' track:', 'Superstar'])
data = sp.search(q, limit=1, offset=0, type='track')
data['tracks']['items']

In [4]:
import re
def getMeta(sp, artist = '', track = '', idn=''):
    '''returns a row of df_meta'''
    if idn:
        try:
            meta = sp.track(idn)
        except:
            return pd.DataFrame.from_dict([{'id':idn,'name':track,'album_name':'','album_id':'',
                                            'artists':artist,'duration_ms':0,'popularity':-1}])
    elif artist or track:
        q=''
        if artist:
            artist = re.sub(r'[\']','',artist)
            q = ''.join([q, 'artist:', artist])
        if track:
            track = re.sub(r'[\']','',track)
            q = ''.join([q, ' track:', track])
        data = sp.search(q, limit=1, offset=0, type='track')['tracks']['items']
        nam = artist.split(' ')
        trk = track.split('(')
        while len(data) < 1: #empty search results
            if len(nam) > 1: #use 1st word of artist name if exists
                q = ''.join(['artist:', nam[0], ' track:', track])
                data = sp.search(q, limit=1, offset=0, type='track')['tracks']['items']
                nam = []
            elif len(trk) > 1: #use track title without '()'
                q = ''.join(['artist:', artist, ' track:', trk[0]])
                data = sp.search(q, limit=1, offset=0, type='track')['tracks']['items']
                trk = []
            else:
                # print(data['tracks']['href'])
                # raise ValueError('getMeta broke for {track} by {artist}.'.format(track=track, artist=artist))
                return pd.DataFrame.from_dict([{'id':idn,'name':track,'album_name':'','album_id':'',
                                                'artists':artist,'duration_ms':0,'popularity':-1}])
        meta = data[0]
    else:
        return pd.DataFrame.from_dict([{'id':idn,'name':track,'album_name':'','album_id':'',
                                        'artists':artist,'duration_ms':0,'popularity':-1}])
    inpt = {}
    inpt['id'] = meta['id']
    inpt['name'] = meta['name']
    inpt['album_name'] = meta['album']['name']
    inpt['album_id'] = meta['album']['id']
    arts = {}
    for i in meta['artists']:
        arts[i['id']]=i['name']
    inpt['artists'] = arts
    inpt['duration_ms'] = meta['duration_ms']
    inpt['popularity'] = meta['popularity']
    outp = pd.DataFrame.from_dict([inpt])
    return outp

In [ ]:
def searchID(sp, q):
    data = sp.search(q, limit=1, offset=0, type='track')['tracks']['items']
    if len(data) > 0:
        return data[0]['id']
    return ''

In [ ]:
# df_hist.to_string()
df_hist.head()

In [ ]:
df_search = df_hist.copy()
print(df_search.shape)
df_search = df_search.drop(['endTime','msPlayed'], axis = 1)
df_search = pd.DataFrame.drop_duplicates(df_search)
print(df_search.shape)

In [11]:
df_hist = pd.read_csv('df_hist.csv', header=0)
df_lib = pd.read_csv('df_lib.csv', header=0)
df_arch = pd.read_csv('df_arch.csv', header=0)
df_search = pd.concat([df_hist.id, df_lib.id, df_arch.id], axis=0)
df_search

0       3pvNllGp0jKVnWo7XNz4U0
1       7eBjtPZlQEDmSJFL5tCkIc
2       2bCjaF7cRm53Wdq9RtkYJu
3       0UhyQ3rTGmSf64TwmKLu3i
4       0y60itmpH0aPKsFiGxmtnh
                 ...          
1369    0Q1nTiEPnoQSqM0qrY9TMY
1370    476Rnxz7dVhH3nQkF7dCDu
1371    0d5OqgwdmMGaJbQOkWWwHI
1372    4lD90LPUMiJIHPRZQ1l2I3
1373    3WC9TbU8v5xRGeSCdaKQw2
Name: id, Length: 22984, dtype: object

In [8]:
#df_meta_hist
df_search = pd.read_csv('df_search.csv', header=0, index_col=None, sep=',', encoding='UTF-8')
df_meta = pd.DataFrame(columns=['id', 'name', 'album_id', 'album_name', 'artists', 'duration_ms', 'popularity'])
# searched = set()
# for row in df_search.itertuples(index=False, name=None):
for row in df_search.id:
    df_meta = pd.concat([df_meta, getMeta(sp, idn=row)], axis = 0)
df_meta.to_csv('df_meta.csv',header=True, sep=',', index=False, encoding='UTF-8')
df_meta.head()

,id,name,album_id,album_name,artists,duration_ms,popularity
0,3pvNllGp0jKVnWo7XNz4U0,Crzy,4XoHw6vA2iF1pv8SuqpsGU,Crzy,"{'0dXCvd6lpLfZBKTFjUF1R3': 'Marius', '5LeUKbWR...",189000,22
0,7eBjtPZlQEDmSJFL5tCkIc,Midnight Racer,39Xf184PU6ESRke6jtAk8n,Escapism III,{'07UJz804RJxqNvxFXC3h9H': 'Sam Gellaitry'},228174,34
0,2bCjaF7cRm53Wdq9RtkYJu,Neck And Neck - DNMO Remix,3W8TE3qqvhwhdGunef80Fe,Northern Lights (Remixes),"{'67qogtRNI0GjUr8PlaG6Zh': 'Zeds Dead', '4GLJP...",182400,59
0,0UhyQ3rTGmSf64TwmKLu3i,I Love It When You Cry (Moxoki) - Boehm Remix,5NK5J3xNka3Zh4MsOioFWP,Ultra Summer 2 (The Best In Deep and Tropical ...,"{'77AiFEVeAVj2ORpC85QVJs': 'Steve Aoki', '1dSR...",322500,35
0,0y60itmpH0aPKsFiGxmtnh,Wait a Minute!,0wfne2JijoxJm0qzJd3V5h,ARDIPITHECUS,{'3rWZHrfrsPBxVy692yAIxF': 'WILLOW'},196520,83


In [ ]:
df_meta.reset_index(drop=True, inplace=True)
# df_meta.drop_duplicates(inplace=True, ignore_index=True)
df_meta.to_csv('df_meta.csv', index=False, encoding='UTF-8')

In [ ]:
#import meta
df_meta = pd.read_csv(r'df_meta.csv', header=0, index_col=None, sep=',', encoding='UTF-8')

In [ ]:
#make df_lib

df_lib = pd.DataFrame.from_dict(jsonToData(lib_file)['tracks'])
# df_lib

In [ ]:
#df_meta_lib
searched = set()
for row in df_lib.itertuples(index=False, name=None):
    a = ''.join([row[0],row[2]])
    if a not in searched:
        searched.add(a)
        df_meta = pd.concat([df_meta, getMeta(sp, row[0], row[2])], ignore_index=True)

In [ ]:
#make archive
# df_archive = j2pd_playlists(pl_file)
# df_archive = pd.DataFrame.from_dict(df_archive['Archive'])
for row in df_archive.itertuples(index=False, name=None):
    a = ''.join([row[1],row[0]])
    if a not in searched:
        searched.add(a)
        df_meta = pd.concat([df_meta, getMeta(sp, row[1], row[0])], ignore_index=True)

In [ ]:
#get -1s and manually check for song id
print(df_meta.shape)


In [ ]:
mask = (df_meta.duration_ms == 0)
df_failed = df_meta.loc[mask, ['name','artists']]
print(df_failed.shape)
df_failed.drop_duplicates(inplace=True, ignore_index=True)
print(df_failed.shape) #(209, 2)
df_failed.to_csv('df_failed.csv', header=True, index=False, encoding='UTF-8')

In [ ]:
#reload df_failed after manually filling out loose ends
df_failed = pd.read_csv('df_failed.csv', header=0, encoding='UTF-8')
df_failed.head()

In [ ]:
#add failed
for row in df_failed.itertuples(index=False, name=None):
    df_meta = pd.concat([df_meta, getMeta(sp, idn=row[2])], ignore_index=True)
print('done')

In [ ]:
#clean meta
#remove failed entries
#remove dupes based on ID
# print(df_meta.shape)
# mask = (df_meta.duration_ms != 0)
# df_meta = df_meta.loc[mask]
print(df_meta.shape)
df_meta.drop_duplicates(subset='id', keep='first', inplace=True)
print(df_meta.shape)

In [ ]:
#acoustic features
df_acoustic = pd.DataFrame(columns=['id',
                                    'danceability',
                                    'energy', 
                                    'key',
                                    'loudness',
                                    'mode',
                                    'speechiness',
                                    'acousticness',
                                    'instrumentalness',
                                    'liveness',
                                    'valence',
                                    'tempo',
                                    'duration_ms',
                                    'time_signature'
                                    ])
for row in df_meta.itertuples(index=False, name=None):
    df_acoustic = pd.concat([df_acoustic, pd.DataFrame.from_dict(sp.audio_features(str(row[0])))], ignore_index=True)
df_acoustic.drop(columns=['type', 'uri', 'track_href', 'analysis_url'],inplace=True)
df_acoustic.drop(columns = [df_acoustic.columns[0]],axis = 1, inplace=True)
df_acoustic.to_csv('df_acoustic.csv', header=True, index=False, encoding='UTF-8')

In [14]:
#acoustic features
df_acoustic = pd.DataFrame(columns=['id',
                                    'danceability',
                                    'energy', 
                                    'key',
                                    'loudness',
                                    'mode',
                                    'speechiness',
                                    'acousticness',
                                    'instrumentalness',
                                    'liveness',
                                    'valence',
                                    'tempo',
                                    'duration_ms',
                                    'time_signature'
                                    ])
df_search = pd.read_csv('df_search.csv', header=0, index_col=None, sep=',', encoding='UTF-8')
c = 0
for row in df_search.id:
    df_acoustic = pd.concat([df_acoustic, pd.DataFrame.from_dict(sp.audio_features(row))], axis=0, ignore_index=True)
# df_acoustic.drop(columns=['type', 'uri', 'track_href', 'analysis_url'],inplace=True)
# df_acoustic.drop(columns = [df_acoustic.columns[0]],axis = 1, inplace=True)
df_acoustic.to_csv('df_acoustic.csv', header=True, index=False, encoding='UTF-8')

In [38]:
#df_analytic
# df_analytic = pd.DataFrame()
# a = sp.audio_analysis('2egKQvoHiCJyHXn8lRlsrl')
print(a.keys())
# print(a['bars']
# print(a['bars']['mode'])
# df_bars = pd.DataFrame.from_dict(a['bars'])
df_beats = pd.DataFrame.from_dict(a['beats'])
df_sections = pd.DataFrame.from_dict(a['sections'])
df_segments = pd.DataFrame.from_dict(a['segments'])
df_tatums = pd.DataFrame.from_dict(a['tatums'])


dict_keys(['meta', 'track', 'bars', 'beats', 'sections', 'segments', 'tatums'])


In [ ]:
#replace artist/track with id
# df_hist.to_csv('df_hist.csv', header=True, index=False, encoding='UTF-8')
#get reduced meta
#reduce hist columns
#merge
df_meta_red = df_meta.copy()
df_meta_red.head()
# df_meta_red['album_track'] = df_meta_red.album_name + ' ' + df_meta_red.name
# df_meta_red.drop(columns = ['name', 'album_id', 'album_name','artists', 'popularity'], inplace=True)

In [19]:
df_hist = pd.read_csv('df_hist.csv', header=0, sep=',', encoding='UTF-8')
# df_hist['q'] = df_hist['name'] + ' ' + df_hist['artist_name']
# df_hist['id'] = df_hist['q'].apply(lambda x: searchID(sp, x))
df_hist.shape

(20725, 6)

In [22]:
df_hist.dropna(axis=0, inplace=True)
df_hist.shape
df_hist.to_csv('df_hist.csv', header=True, sep=',', index=False, encoding='UTF-8')